# WallSlave

**Autor:** Breno Araújo Junqueira - Wineano

**Descrição:** Esse é um notebook para automatização de transferências de itens entre filiais pela plataforma Wall da Wine. Os scripts são escritos em Python e utiliza uma API da Selenium para automatização de aplicações na web. Essa ferramenta nos permite simular cliques e digitações em um website HTML. 

Para seu uso, devem ser seguidos os passos abaixo:


## Passo 0:

Faça o download do Webdriver do seu navegador. Aqui mostrarei usando o chrome. Link para download do lastest stable release: https://chromedriver.chromium.org/. Cole o aplicativo instalado no C:\Program Files (x86)

## Passo 1:

Rode a célula abaixo para instalar a API Selenium.

In [3]:
!pip install -U selenium

     |████████████████████████████████| 911kB 2.9MB/s 


## Passo 2:

Rode as célula abaixo para instalar bibliotecas e importar o arquivo Excel com os dados de transferência a partir da caixa de diálogo que aparecerá.

Para correto funcionamento do código, o arquivo Excel deve conter as seguintes características:

* As abas devem conter o nome "Loja" para solicitações da loja Wine e "VF" para solicitções da Vinho Fácil.
* A primeira coluna deve conter os SKUs
* A segunda coluna deve conter as descrições
* A terceira coluna deve conter as quantidades solicitadas
* As demais colunas são opcionais e não serão usadas no código.

O modelo de planilha pode ser encontrado em: https://drive.google.com/file/d/1I39SB6dakO7eI_JvOHeMTCOCo6G1VUNb/view?usp=sharing

Seleciona a planilha de excel com as transferências desejadas.

In [24]:
import pandas as pd
from google.colab import files
uploaded = files.upload()
import io
filename = list(uploaded)[0]    

Saving TransfsCOMEX06.05.xlsx to TransfsCOMEX06.05 (2).xlsx


## Passo 3: 

Rode a célula abaixo e escreva o nome da Aba (Loja ou VF) que deseja inserir no wall. Pressione Enter para concluir

In [29]:
print('SELECIONE A LOJA DESTINO:')  #digite VF para Vinho Fácil ou Loja para Loja Wine
loja = input()
df = pd.read_excel(filename,sheet_name=loja)

SELECIONE A LOJA DESTINO:
Loja


## Passo 4:
 Rode a célula abaixo para a verificar a lista de itens e quantidades inserida.

In [28]:
df

,SKU,Rótulo,QNT.,1- 1 COMEX,DESFLEGAR CLUBE,p\ LOJA,flegar clube
0,25020,1922 Blend 2019,48,NaN,NaN,NaN,NaN
1,25021,1922 Bonarda 2020,48,NaN,NaN,NaN,NaN
2,25019,1922 Syrah Malbec 2020,246,NaN,NaN,NaN,NaN
3,25798,370 Léguas Reserva D.O.C Douro 2017,36,NaN,NaN,NaN,NaN
4,24888,Abridor Blend 2019,180,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
259,23500,Velar Primitivo Di Manduria Doc 2016,36,NaN,NaN,NaN,NaN
260,24168,Viña Bouchon País Salvaje 2019,2,NaN,NaN,NaN,NaN
261,25008,Viña Labrada D.O.P. Cariñena Garnacha 2019,120,NaN,NaN,NaN,NaN
262,25010,Viña Labrada D.O.P. Cariñena Tempranillo 2019,180,NaN,NaN,NaN,NaN


## Passo 5:

 Rode a célula abaixo para importar as APIs Selenium necessárias

In [30]:
import time
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## Passo 6: 

Rode a célula abixo para abrir a nova janela controlada pela API onde será realizada a automação. Se necessário entrar com sua conta google. O passo 0 deve ser coretamente seguido para proseguimento dessa etapa.

In [31]:
#ABRE PÁGINA DO WALL
driver = webdriver.Chrome(executable_path="C:\Program Files (x86)\chromedriver2")
driver.get("https://wall.wine.com.br/index.html")

time.sleep(7)

try:
    element = WebDriverWait(driver,20).until(
        EC.presence_of_element_located((By.ID,"signin-button"))
    )
finally:
    #CLICANDO P/ LOGAR NO GOOGLE - EMAIL E SENHA MANUAIS
    login = driver.find_element_by_id("signin-button")
    login.send_keys(Keys.RETURN)




WebDriverException: ignored

## Passo 7: 
Rode a célula abixo para abrir o menu literal e selecionar transferência de estoque. Essa etapa pode dar erro. Caso acontesa, faça-a manualmente com se fosse transferir manualmente.

In [ ]:
#OPERANDO NO MENU LATERAL: CADASTRO>>TRANSFERÊNCIA DE ESTOQUE - MANUAL
cadastrotreeview = driver.find_element_by_tag_name("a")
cadastrotreeview.send_keys(Keys.RETURN)

testoque= driver.find_element_by_xpath('//*[@id="menu"]/ul/li[2]/ul/li/a')
###############Ahhhhhhhhhhhhhh   testoque.send_keys(Keys.RETURN)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"a"}
  (Session info: chrome=90.0.4430.212)


## Passo 8: 
Rode a célula abixo para abrir inserir a origem, destino e canal da trasnferência (Passo 1 do Wall).

In [ ]:
#ESCOLHENDO ORIGENS E DESTINOS
filialO = driver.find_element_by_id("cbx-company-from")
filialO.send_keys("[5] -")
filialO.send_keys(Keys.RETURN)

time.sleep(2)

filialD = driver.find_element_by_id("cbx-company-to")
filialD.send_keys(Keys.ARROW_DOWN)
filialD.send_keys(Keys.RETURN)

time.sleep(2)

almoxD = driver.find_element_by_id("cbx-store")
almoxD.send_keys(Keys.ARROW_DOWN)
if loja =="VF":
    almoxD.send_keys(Keys.ARROW_DOWN)
almoxD.send_keys(Keys.RETURN)

time.sleep(2)

proximo= driver.find_element_by_link_text("Próximo")
proximo.send_keys(Keys.RETURN)


## Passo 9: 
Esse é o passo mais importante e de fato o que vai economizar seu tempo. Rode a célula abixo para executar o looping de comandos que opera o wall conforme a lista aqui inserida.

In [ ]:
size = len(df)

for i in range(int(size)): 
    inserir = driver.find_element_by_tag_name("button")
    inserir.click()
    
    time.sleep(3)
    
    sku = df.iloc[i][0]  
    qnt = df.iloc[i][2] 
    
    camposeta = driver.find_element_by_xpath('//*[@id="formDados"]/div/div[2]/div[1]/div[1]/div/span[1]')
    camposeta.click()
    
    camposku = driver.find_element_by_class_name("select2-search__field")
    camposku.send_keys(str(int(sku)))
    time.sleep(4)
    camposku.send_keys(Keys.RETURN)
    time.sleep(4)
    campoqnt = driver.find_element_by_id("txt-quantity")
    campoqnt.send_keys(str(int(qnt)))
    time.sleep(4)
    campoqnt.send_keys(Keys.RETURN)
    time.sleep(4)
    
    
    

## OBSERVAÔES IMPORTANTES:

Alguns erros podem ocorrer se:
- Houver saldo insuficiente;
- O item estiver com flag desabilitado ou estiver com flag do clube;
- Se a página do Wall demorar mais que o tempo entre comandos determinado. Esse tempo entre comandos pode ser modificado conforme sua necessidade alterando o número de segundos nas funções "time.sleep()" do script acima. Mas lembre que, se for mais curto que o carregamento do Wall teremos erro.


In [ ]:
driver.quit()